In [1]:
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import codecs
mypath ="/Anaconda/blogs"
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from time import time
import sys
import scipy.sparse as sp
import pylab as pl
import cPickle

In [2]:
from sklearn.datasets import load_mlcomp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [3]:
def benchmark(clf_class, params, name):
    print("parameters:", params)
    t0 = time()
    clf = clf_class(**params).fit(Xtrain, y_train)
    print("done in %fs" % (time() - t0))

    if hasattr(clf, 'coef_'):
        print("Percentage of non zeros coef: %f"
              % (np.mean(clf.coef_ != 0) * 100))
    print("Predicting the outcomes of the testing set")
    t0 = time()
    pred = clf.predict(Xtest)
    print("done in %fs" % (time() - t0))

    print("Classification report on test set for classifier:")
    print(clf)
    print()
    print(classification_report(y_test, pred))
    
    with open(name+'.pkl', 'wb') as fid:
        cPickle.dump(clf, fid) 
    
    cm = confusion_matrix(y_test, pred)
    print("Confusion matrix:")
    print(cm)

    # Show confusion matrix
    pl.matshow(cm)
    pl.title('Confusion matrix of the %s classifier' % name)
    pl.colorbar()


In [4]:
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [5]:
def make_documentIndex(listOfDocuments):
    dict_of_words = {}
    doc_corpus = []
    for doc in listOfDocuments:           
        file = open(mypath+"/"+doc)
        str = file.read()
        doc_corpus.append(str)

        
       
    return doc_corpus   

In [6]:
corpus = make_documentIndex(onlyfiles)

In [7]:
len(corpus)

19320

In [8]:
def HandleDataAndLabels(coprus,onlyfiles):
    Y = []
    X = []
    numFiles = len(onlyfiles)
    for f in range(0,numFiles):
        age = onlyfiles[f].split('.')[2]
        ag = 0
        if int(age) <= 17:
            ag= 1
        elif int(age) >17 and int(age)< 33:
            ag= 2
        elif int(age) >=33:
            ag= 3
     
        soup = BeautifulSoup(corpus[f],"lxml")
        result= soup.findAll('post')
        for a in result:
            X.append(a.get_text())
            Y.append(ag)
            
    return X,Y


In [9]:
X,Y = HandleDataAndLabels(corpus,onlyfiles)
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7)


In [10]:
print "Original Data X,Y",len(X),len(Y)
print "Xtrain,Ytrain ",len(X_train),len(y_train)
print "XTest, YTest ",len(X_test),len(y_test)

Original Data X,Y 681253 681253
Xtrain,Ytrain  476877 476877
XTest, YTest  204376 204376


In [11]:
t0 = time()
vectorizer = TfidfVectorizer(encoding='utf-8')
Xtrain = vectorizer.fit_transform(X_train)
print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % Xtrain.shape)
assert sp.issparse(Xtrain)


done in 116.214000s
n_samples: 476877, n_features: 586663


In [12]:
with open('TFIDF_Vectorizer.pkl', 'wb') as fid:
    cPickle.dump(vectorizer, fid)    


In [13]:

t0 = time()
Xtest = vectorizer.transform(X_test)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % Xtest.shape)


done in 46.528000s
n_samples: 204376, n_features: 586663


In [14]:
t0 = time()
clf = RandomForestClassifier(n_estimators=100,max_depth=32,n_jobs=4,verbose=True)
clf.fit(Xtrain, y_train)
print("done in %fs" % (time() - t0))

if hasattr(clf, 'coef_'):
    print("Percentage of non zeros coef: %f"
          % (np.mean(clf.coef_ != 0) * 100))
print("Predicting the outcomes of the testing set")
t0 = time()
pred = clf.predict(Xtest)
print("done in %fs" % (time() - t0))

print("Classification report on test set for classifier:")
print(clf)
print()
print(classification_report(y_test, pred))

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  6.5min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    5.7s finished


done in 395.579000s
Predicting the outcomes of the testing set
done in 6.061000s
Classification report on test set for classifier:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=32, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=True,
            warm_start=False)
()
             precision    recall  f1-score   support

          1       0.92      0.23      0.36     70783
          2       0.51      0.99      0.67     96334
          3       0.97      0.01      0.01     37259

avg / total       0.74      0.54      0.44    204376



In [19]:
"""""
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(Xtrain.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(Xtrain.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()
"""""

'""\nimportances = clf.feature_importances_\nstd = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)\nindices = np.argsort(importances)[::-1]\n\n# Print the feature ranking\nprint("Feature ranking:")\n\nfor f in range(Xtrain.shape[1]):\n    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))\n\n# Plot the feature importances of the forest\nplt.figure()\nplt.title("Feature importances")\nplt.bar(range(Xtrain.shape[1]), importances[indices],\n       color="r", yerr=std[indices], align="center")\nplt.xticks(range(X.shape[1]), indices)\nplt.xlim([-1, X.shape[1]])\nplt.show()\n'

In [15]:

with open('RF_classifier2.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)    

# load it again
#with open('my_dumped_classifier.pkl', 'rb') as fid:
 #   gnb_loaded = cPickle.load(fid)

In [14]:
print("Testbenching a MultinomialNB classifier...")
parameters = {'alpha': 0.01}

benchmark(MultinomialNB, parameters, 'MultiNB')
pl.show()


Testbenching a MultinomialNB classifier...
('parameters:', {'alpha': 0.01})
done in 0.900000s
Percentage of non zeros coef: 100.000000
Predicting the outcomes of the testing set
done in 0.338000s
Classification report on test set for classifier:
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
()
             precision    recall  f1-score   support

          1       0.80      0.68      0.73     70564
          2       0.64      0.85      0.73     96614
          3       0.64      0.30      0.40     37198

avg / total       0.70      0.69      0.67    204376

Confusion matrix:
[[47665 21730  1169]
 [ 9799 81674  5141]
 [ 1931 24221 11046]]


In [15]:
print("Testbenching a linear classifier...")
parameters = {
    'loss': 'hinge',
    'penalty': 'l2',
    'n_iter': 50,
    'alpha': 0.00001,
    'fit_intercept': True,
}

benchmark(SGDClassifier, parameters, 'SGD')
pl.show()

Testbenching a linear classifier...
('parameters:', {'penalty': 'l2', 'loss': 'hinge', 'alpha': 1e-05, 'fit_intercept': True, 'n_iter': 50})
done in 51.508000s
Percentage of non zeros coef: 71.303162
Predicting the outcomes of the testing set
done in 0.275000s
Classification report on test set for classifier:
SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
()
             precision    recall  f1-score   support

          1       0.77      0.71      0.74     70805
          2       0.63      0.86      0.73     96773
          3       0.76      0.15      0.26     36798

avg / total       0.70      0.68      0.65    204376

Confusion matrix:
[[50447 19957   401]
 [11953 83416  1404]
 [ 2929 28166  5703]]


In [16]:
print("Testbenching a logistic Regression...")
parameters = {
    'loss': 'log',
    'penalty': 'l2',
    'n_iter': 250,
    'alpha': 0.000001,
    'fit_intercept': True,
    'verbose':True,
}

benchmark(SGDClassifier, parameters, 'SGD')
pl.show()

Testbenching a logistic Regression...
('parameters:', {'loss': 'log', 'verbose': True, 'fit_intercept': True, 'n_iter': 250, 'penalty': 'l2', 'alpha': 1e-06})
-- Epoch 1
Norm: 335.66, NNZs: 589993, Bias: -0.737125, T: 476877, Avg. loss: 0.611524
Total training time: 0.42 seconds.
-- Epoch 2
Norm: 297.43, NNZs: 589993, Bias: -0.877073, T: 953754, Avg. loss: 0.485343
Total training time: 0.84 seconds.
-- Epoch 3
Norm: 286.99, NNZs: 589993, Bias: -0.780909, T: 1430631, Avg. loss: 0.437168
Total training time: 1.26 seconds.
-- Epoch 4
Norm: 282.28, NNZs: 589993, Bias: -0.805562, T: 1907508, Avg. loss: 0.411354
Total training time: 1.68 seconds.
-- Epoch 5
Norm: 279.50, NNZs: 589993, Bias: -0.838697, T: 2384385, Avg. loss: 0.395094
Total training time: 2.10 seconds.
-- Epoch 6
Norm: 277.59, NNZs: 589993, Bias: -0.861567, T: 2861262, Avg. loss: 0.383832
Total training time: 2.52 seconds.
-- Epoch 7
Norm: 276.54, NNZs: 589993, Bias: -0.794518, T: 3338139, Avg. loss: 0.375537
Total training ti

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.5min finished


In [18]:
print x.shape

(681253, 724796)
